In [1]:
import pandas as pd
from pandasql import sqldf, load_meat, load_births
pysqldf = lambda q: sqldf(q)
import numpy as np

In [5]:
!pip install duckdb


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [8]:
# Get the top-3 busiest train stations
import duckdb
duckdb.sql("""
    SELECT *
    FROM df
    """)

┌─────────────────────┬──────────┬─────────┐
│        Date         │ Expenses │  Type   │
│    timestamp_ns     │  double  │ varchar │
├─────────────────────┼──────────┼─────────┤
│ 2022-01-01 00:00:00 │    143.5 │ Gift    │
│ 2022-01-01 00:00:00 │     25.0 │ Clothes │
│ 2022-01-01 00:00:00 │     NULL │ Gaming  │
│ 2022-01-01 00:00:00 │     24.0 │ Clothes │
│ 2022-01-02 00:00:00 │     3.75 │ Coffee  │
│ 2022-01-05 00:00:00 │     20.0 │ Gift    │
│ 2022-01-06 00:00:00 │      5.0 │ Clothes │
│ 2022-01-07 00:00:00 │    100.0 │ Gaming  │
│ 2022-01-08 00:00:00 │     24.5 │ Clothes │
│ 2022-01-09 00:00:00 │      2.0 │ Coffee  │
│ 2022-01-10 00:00:00 │    100.0 │ Book    │
├─────────────────────┴──────────┴─────────┤
│ 11 rows                        3 columns │
└──────────────────────────────────────────┘

In [20]:
# Get the top-3 busiest train stations
import duckdb
duckdb.sql("""
    SELECT CONCAT('jeje', Type)
    FROM df
    """)

┌────────────────────────┐
│ concat('jeje', "Type") │
│        varchar         │
├────────────────────────┤
│ jejeGift               │
│ jejeClothes            │
│ jejeGaming             │
│ jejeClothes            │
│ jejeCoffee             │
│ jejeGift               │
│ jejeClothes            │
│ jejeGaming             │
│ jejeClothes            │
│ jejeCoffee             │
│ jejeBook               │
├────────────────────────┤
│        11 rows         │
└────────────────────────┘

# Defining the sample (and dummy) table
We generate some dummy data to work with.

In [4]:
# Define the data for the DataFrame
data = {
    'Date': ['1/1/22', '1/1/22', '1/1/22', '1/1/22', '2/1/22','5/1/22', '6/1/22', '7/1/22', '8/1/22', '9/1/22', '10/1/22'],
    'Expenses': [143.5, 25, np.nan, 24, 3.75, 20, 5, 100, 24.5, 2, 100],
    'Type': ['Gift', 'Clothes', 'Gaming', 'Clothes', 'Coffee', 'Gift', 'Clothes', 'Gaming', 'Clothes', 'Coffee', "Book"]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')

df


,Date,Expenses,Type
0,2022-01-01,143.50,Gift
1,2022-01-01,25.00,Clothes
2,2022-01-01,NaN,Gaming
3,2022-01-01,24.00,Clothes
4,2022-01-02,3.75,Coffee
5,2022-01-05,20.00,Gift
6,2022-01-06,5.00,Clothes
7,2022-01-07,100.00,Gaming
8,2022-01-08,24.50,Clothes
9,2022-01-09,2.00,Coffee


In [3]:
# Generate an extended dataframe with day, month and year. 
df_ext = df

# Extract the 'Day' and 'Year' from the 'Date' column
df_ext['Day'] = df['Date'].dt.day
df_ext['Month'] = df['Date'].dt.month
df_ext['Year'] = df['Date'].dt.year

# Create a DataFrame
df_ext

,Date,Expenses,Type,Day,Month,Year
0,2022-01-01,143.50,Gift,1,1,2022
1,2022-01-01,25.00,Clothes,1,1,2022
2,2022-01-01,NaN,Gaming,1,1,2022
3,2022-01-01,24.00,Clothes,1,1,2022
4,2022-01-02,3.75,Coffee,2,1,2022
5,2022-01-05,20.00,Gift,5,1,2022
6,2022-01-06,5.00,Clothes,6,1,2022
7,2022-01-07,100.00,Gaming,7,1,2022
8,2022-01-08,24.50,Clothes,8,1,2022
9,2022-01-09,2.00,Coffee,9,1,2022


## COALESCE
**COALESCE**: It returns the first non-null expression in a list.


In [4]:
pysqldf(""" 
        SELECT *,
              COALESCE(Expenses, 0) AS Expenses_mod
        FROM df""")

,Date,Expenses,Type,Day,Month,Year,Expenses_mod
0,2022-01-01 00:00:00.000000,143.50,Gift,1,1,2022,143.50
1,2022-01-01 00:00:00.000000,25.00,Clothes,1,1,2022,25.00
2,2022-01-01 00:00:00.000000,NaN,Gaming,1,1,2022,0.00
3,2022-01-01 00:00:00.000000,24.00,Clothes,1,1,2022,24.00
4,2022-01-02 00:00:00.000000,3.75,Coffee,2,1,2022,3.75
5,2022-01-05 00:00:00.000000,20.00,Gift,5,1,2022,20.00
6,2022-01-06 00:00:00.000000,5.00,Clothes,6,1,2022,5.00
7,2022-01-07 00:00:00.000000,100.00,Gaming,7,1,2022,100.00
8,2022-01-08 00:00:00.000000,24.50,Clothes,8,1,2022,24.50
9,2022-01-09 00:00:00.000000,2.00,Coffee,9,1,2022,2.00


## LAG
**LAG**: It retrieves data from previous rows over the columns we indicate. 


In [5]:
pysqldf(""" 
        SELECT *,
              LAG(Expenses) OVER(PARTITION BY Type ORDER BY Date)
        FROM df""")

,Date,Expenses,Type,Day,Month,Year,LAG(Expenses) OVER(PARTITION BY Type ORDER BY Date)
0,2022-01-10 00:00:00.000000,100.00,Book,10,1,2022,NaN
1,2022-01-01 00:00:00.000000,25.00,Clothes,1,1,2022,NaN
2,2022-01-01 00:00:00.000000,24.00,Clothes,1,1,2022,25.00
3,2022-01-06 00:00:00.000000,5.00,Clothes,6,1,2022,24.00
4,2022-01-08 00:00:00.000000,24.50,Clothes,8,1,2022,5.00
5,2022-01-02 00:00:00.000000,3.75,Coffee,2,1,2022,NaN
6,2022-01-09 00:00:00.000000,2.00,Coffee,9,1,2022,3.75
7,2022-01-01 00:00:00.000000,NaN,Gaming,1,1,2022,NaN
8,2022-01-07 00:00:00.000000,100.00,Gaming,7,1,2022,NaN
9,2022-01-01 00:00:00.000000,143.50,Gift,1,1,2022,NaN


## RANK
The rank functions numerate the records of a table with a specific order. The way they deal with ties makes us have three different commands to do so:
- **RANK**: Deals with ties, giving the same number to records with the same value, leaving gaps after them. 
- **ROW_NUMBER**: Give unique numbers to all records. If the value is the same, it just gives a random order. 
- **DENSE_RANK**: Deals with ties, giving the same number to records with the same value, without leaving gaps after them. 


In [8]:
df_mod = pysqldf(""" 
        SELECT Type, 
                 SUM(Expenses) AS Expenses
        FROM df
        GROUP BY 1
        ORDER BY 2 DESC""")

df_mod


,Type,Expenses
0,Gift,163.50
1,Gaming,100.00
2,Book,100.00
3,Clothes,78.50
4,Coffee,5.75


In [12]:
pysqldf(""" 
        SELECT *, 
               --RANK() OVER (ORDER BY Expenses DESC) AS Rank,
               --ROW_NUMBER() OVER (ORDER BY Expenses DESC) AS Row_Number
               DENSE_RANK() OVER (ORDER BY Expenses DESC) AS Dense_Rank
        FROM df_mod""")

,Type,Expenses,Dense_Rank
0,Gift,163.50,1
1,Gaming,100.00,2
2,Book,100.00,2
3,Clothes,78.50,3
4,Coffee,5.75,4
